In [1]:
from sentence_transformers import SentenceTransformer
from datetime import datetime

import logging
import re

import numpy as np
import pandas as pd

import hdbscan
import umap

import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
model_path = "kpfsbert-base" # sbert
cluster_mode = 'title'

In [4]:
# Data Loading to clustering
DATA_PATH = 'data/newstrust/newstrust_20210601_samlple.json'
df = pd.read_json(DATA_PATH, encoding='utf-8')
len(df)

12006

In [5]:
#kpfSBERT 모델 로딩
model = SentenceTransformer(model_path)

In [6]:
# UMAP 차원축소 실행
def umap_process(corpus_embeddings, n_components=5):
    umap_embeddings = umap.UMAP(n_neighbors=15, 
                                n_components=n_components, 
                                metric='cosine').fit_transform(corpus_embeddings)
    return umap_embeddings

In [7]:
# HDBSCAN 실행
def hdbscan_process(corpus, corpus_embeddings, min_cluster_size=15, min_samples=10, umap=True, n_components=5, method='eom'):
    
    if umap:
        umap_embeddings = umap_process(corpus_embeddings, n_components)
    else:
        umap_embeddings = corpus_embeddings
    
    cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,
                              min_samples=10,
                              allow_single_cluster=True,
                              metric='euclidean',
                              core_dist_n_jobs=1,# knn_data = Parallel(n_jobs=self.n_jobs, max_nbytes=None) in joblib
                              cluster_selection_method=method).fit(umap_embeddings) #eom leaf

    docs_df = pd.DataFrame(corpus, columns=["Doc"])
    docs_df['Topic'] = cluster.labels_
    docs_df['Doc_ID'] = range(len(docs_df))
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})
    
    return docs_df, docs_per_topic

In [8]:
# 카테고리별 클러스터링
start = datetime.now()
print('작업 시작시간 : ', start)
previous = start
bt_prev = start

tot_df = pd.DataFrame()

print(' processing start... with cluster_mode :', cluster_mode)

category = df.category.unique()

df_category = []
for categ in category:
    df_category.append(df[df.category==categ])
cnt = 0
rslt = []
topics = []
#순환하며 데이터 만들어 df에 고쳐보자
for idx, dt in enumerate(df_category):

    corpus = dt[cluster_mode].values.tolist()
    # '[보통공통된꼭지제목]' 형태를 제거해서 클러스터링시 품질을 높인다.
    for i, cp in enumerate(corpus):
        corpus[i] = re.sub(r'\[(.*?)\]', '', cp)
#     print(corpus[:10])
    corpus_embeddings = model.encode(corpus, show_progress_bar=True)

    docs_df, docs_per_topic = hdbscan_process(corpus, corpus_embeddings,
                                               umap=False, n_components=15, #연산량 줄이기 위해 umap 사용시 True 
                                               method='leaf',
                                               min_cluster_size=5,
                                               min_samples=30,
                                               )
    cnt += len(docs_df)

    rslt.append(docs_df)
    topics.append(docs_per_topic)
    dt['cluster'] = docs_df['Topic'].values.tolist()
    tot_df = pd.concat([tot_df,dt])

    bt = datetime.now()
    print(len(docs_df), 'docs,', len(docs_per_topic)-1 ,'clusters in', category[idx], ', 소요시간 :', bt - bt_prev)
    bt_prev = bt
now = datetime.now()
print(' Total docs :', cnt,'in', len(rslt), 'Categories', ', 소요시간 :', now - previous)
previous = now

#cluster update    

df['cluster'] = tot_df['cluster'].astype(str)
    
end = datetime.now()
print('작업 종료시간 : ', end, ', 총 소요시간 :', end - start)

작업 시작시간 :  2021-12-13 14:57:41.800864
 processing start... with cluster_mode : title


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

819 docs, 10 clusters in IT 과학 , 소요시간 : 0:00:02.874309


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

2662 docs, 38 clusters in 경제 , 소요시간 : 0:00:09.710102


...

259 docs, 2 clusters in 교육 , 소요시간 : 0:00:00.291253


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

79 docs, 0 clusters in 라이프스타일 , 소요시간 : 0:00:00.094746


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

266 docs, 3 clusters in 스포츠 , 소요시간 : 0:00:00.310171
 Total docs : 12006 in 11 Categories , 소요시간 : 0:00:46.768052
작업 종료시간 :  2021-12-13 14:58:28.573904 , 총 소요시간 : 0:00:46.773040


In [9]:
categ = '사회'
condition = (df.category == categ) & (df.cluster == '2') # 조건식 작성

In [10]:
test = df[condition]
print(len(test))
test.title.values.tolist()

12


['사제 총 만들어 판 일당 적발…한국도 총기 안전지대 아니다',
 '美서 총기부품 위장 수입… 총기 제작·판매 7명 적발',
 '부품 밀수해 총기 만들어 판매… 현역 군인 포함한 일당 검거',
 "부품 들여와 진짜 권총 만들었다, 300만원에 판 '고스트 건' [영상]",
 '불법 수입한 부품 이용해 권총·소총 제작·판매한 현역 군인 포함 7명 검거',
 '부품 밀수해 총기 제작…현직 군인도 연루',
 '부품 밀수해 총기 제작…현직 군인도 연루',
 '경찰, 해외서 부품 수입해 총기류 제조·판매 한 일당 적발',
 "미국 총기 난사 '고스트건'…부품 밀수 후 국내서 총기 제작 판매 적발",
 '부품 밀수해 총기 제조...SNS로 판매까지',
 '부품 밀수해 국내서 총기 제조·판매...6정 적발',
 '진짜 총 만들어 판 총기 동호회원 적발…부품 밀수한 ‘고스트건’']

In [11]:
# 클러스터별 주제어 추출확인
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [12]:
# 카테고리별 클러스터별 주제어 추출확인
category_id = 3

In [13]:
tf_idf, count = c_tf_idf(topics[category_id].Doc.values, m=len(corpus))

top_n_words = extract_top_n_words_per_topic(tf_idf, count, topics[category_id], n=20)
topic_sizes = extract_topic_sizes(rslt[category_id]); topic_sizes.head(10)

,Topic,Size
0,-1,3811
35,34,128
40,39,66
29,28,50
42,41,44
39,38,43
41,40,33
16,15,33
12,11,30
32,31,27


In [14]:
len(topic_sizes)

54

In [15]:
top_n_words[2][:10]

[('부품', 0.31853506949394694),
 ('총기', 0.3008339474751164),
 ('밀수해', 0.19291545613337857),
 ('적발', 0.16021065813934854),
 ('판매', 0.1504169737375582),
 ('제작', 0.12561961336056837),
 ('제조', 0.12224860041733422),
 ('만들어', 0.11043893707437799),
 ('일당', 0.10205984758549116),
 ('고스트건', 0.09495823549945152)]